In [1]:
import pandas as pd
import numpy as np
import re
import os
import nltk
from nltk.corpus import stopwords
from nltk.tag import CRFTagger
from nltk.tokenize import sent_tokenize, word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [6]:
df = pd.read_csv('train.csv')
df

,id,sumber,kodekat,kodesubkat,kategori,subkategori,katakunci,tanggal,judul_berita,konten_berita,nama_tokoh,jabatan,organisasi,lokasi,alias,kutipan,sentimen_kutipan,sentimen_berita
0,00e3395ef29,Antara,J,J,Informasi dan Komunikasi,Informasi dan Komunikasi,Laporan keuangan,2021-02-05,ASN penyeleweng dana infak Masjid Raya divonis...,Padang (ANTARA) - Oknum Aparatur Sipil Negara ...,['Rinto'],['Hakim Ketua Yose'],[],"['Padang', 'Sumbar', 'Provinsi Sumbar', 'Inspe...","['dibayar paling lama', '¬©']","['""Menyatakan terdakwa terbukti secara sah dan...",Negatif,Negatif
1,019a47ed0bc,Detik,A,A2,"Pertanian, Kehutanan, dan Perikanan",Kehutanan dan Penebangan Kayu,hasil hutan,2021-01-19,10 Alasan MK Kategorikan Ganja Hidroponik seba...,Jakarta -\n\nMahkamah Konstitusi (MK) memasukk...,"['Dengler', 'Recommended Methods For', 'I']",[],"['Mahkamah Konstitusi (MK)', 'MK', 'KBBI: poho...","['Jakarta', 'Yogyakarta', 'Manual', '>>']",[],[],Netral,Negatif
2,01eb3258ed4,Antara,D,D1,Pengadaan Listrik dan Gas,Ketenagalistrikan,Listrik PLN,2021-03-31,Angkasa Pura minta maaf atas mati listrik di ...,Padang (ANTARA) - PT Angkasa Pura II selaku pe...,"['Ikhwan Wahyudi', 'Adi Lazuardi']",[],"['PT Angkasa Pura II', 'PLN', 'BIM', '***1***']","['Padang', 'Sumbar', 'Kota Padang', 'Kabupaten...",['¬©'],"['""Kami menyampaikan permohonan maaf yang sebe...",Negatif,Negatif
3,02319ba7dbc,Okezone,J,J,Informasi dan Komunikasi,Informasi dan Komunikasi,Jumlah Penonton,2021-05-07,"Liga 1 2021 Digelar dengan Kehadiran Penonton,...","JAKARTA ‚Äì Direktur Utama PT LIB, Akhmad Hadi...","['Akhmad Hadian Lukita', 'Akhmad', 'Akhmad Had...","['Direktur Utama PT LIB', 'Presiden']","['The Jakmania', 'Kementerian Pemuda dan Olahr...","['Indonesia', 'Jakarta']",[],"['""Kajian terus kami lakukan dan itu tidak gam...",Netral,Negatif
4,026dd5917f6,Detik,J,J,Informasi dan Komunikasi,Informasi dan Komunikasi,Laporan keuangan,2021-01-23,"Bill Gates Kuasai Tanah Pertanian, Netizen Ket...",Jakarta -\n\nAda hal baru yang mungkin belum b...,"['Offut', 'Gates', 'Michael Larson', 'Donald T...",[],[],"['Jakarta', 'Amerika Serikat', 'Amerika', 'Afr...",[],"['""Tak boleh ada seseorang atau perusahaan bis...",Negatif,Negatif
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52884,b79dee8f99d,Antara,K,K1,Jasa Perantara Keuangan dan Asuransi,Jasa Perantara Keuangan,kredit,2021-05-25,"BRI: Total aset Rp1.411,05 triliun kuartal I 2...",Kami akan terus menjadi mitra pemerintah dalam...,['Utama BRI Sunarso dalam'],['Direktur Utama BRI'],"['PT Bank Rakyat Indonesia (Persero) Tbk', 'BRI']","['Jakarta', 'Indonesia', 'Sumatera Utara']",['¬©'],"['""Total aset mencapai Rp1.411,05 triliun yang...",Positif,Positif
52885,b7a07fc1f97,Antara,J,J,Informasi dan Komunikasi,Informasi dan Komunikasi,teknologi,2021-05-12,IGJ apresiasi keputusan Pemerintah RI dorong s...,Bila proposal itu disetujui diharapkan akan le...,"['Waiver', 'Rachmi Hertanti', 'Rachmi', 'Rahma...",['Direktur Eksekutif IGJ'],"['Global Justice (IGJ)', 'Pemerintah RI', 'Pem...","['Jakarta', 'Indonesia', 'Afrika Selatan', 'In...",[],"['""Keputusan pemerintah menjadi co-sponsor mer...",Positif,Positif
52886,b7a0872bb7e,Detik,G,G1,Perdagangan Besar dan Eceran; Reparasi Mobil d...,Perdagangan Mobil dan Sepeda Motor dan Reparas...,penjualan mobil,2021-06-29,Penantang Avanza Ini Harganya Lebih Murah dari...,Jakarta -\n\nWuling Confero lebih dulu mengumu...,[],[],[],"['Jakarta', 'China', 'indonesia']",[],"['""Wuling Confero S Facelift Resmi Diluncurkan']",Positif,Positif
52887,b7a182ae9f4,Detik,R,R,"Jasa Kesenian, Hiburan dan Rekreasi Swasta","Jasa Kesenian, Hiburan dan Rekreasi Swasta",konser musik,2021-05-05,"Kerumunan Tanah Abang-Konser, Doni Singgung 40...","Jakarta -\n\nKetua Satgas COVID-19, Doni Monar...","['Doni Monardo', 'Doni']",['Ketua Satgas COVID-19'],['BPS'],"['Jakarta', 'Jakarta Selatan', 'India', 'Catat...",[],"['""Memang ada persoalan yang kita hadapi karen...",Positif,Positif


In [3]:
df_content = df.loc[:, ['konten_berita', 'sentimen_berita']]
df_content['konten_berita'] = df_content['konten_berita'].str.lower()
df_content

,konten_berita,sentimen_berita
0,padang (antara) - oknum aparatur sipil negara ...,Negatif
1,jakarta -\n\nmahkamah konstitusi (mk) memasukk...,Negatif
2,padang (antara) - pt angkasa pura ii selaku pe...,Negatif
3,"jakarta ‚äì direktur utama pt lib, akhmad hadi...",Negatif
4,jakarta -\n\nada hal baru yang mungkin belum b...,Negatif
...,...,...
52884,kami akan terus menjadi mitra pemerintah dalam...,Positif
52885,bila proposal itu disetujui diharapkan akan le...,Positif
52886,jakarta -\n\nwuling confero lebih dulu mengumu...,Positif
52887,"jakarta -\n\nketua satgas covid-19, doni monar...",Positif


In [4]:
df_content['konten_berita'][0]

'padang (antara) - oknum aparatur sipil negara (asn) pada pemprov sumatera barat (sumbar) yelnazi rinto yang menjadi terdakwa kasus penyelewengan uang infak masjid raya sumbar serta korupsi sejumlah anggaran lain divonis dengan hukuman tujuh tahun penjara.\n\n"menyatakan terdakwa terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana korupsi secara berlanjut, menjatuhkan pidana selama tujuh tahun," kata hakim ketua yose ana roslinda dalam amar yang dibacakan di padang, jumat.\n\namar tersebut dibacakan hakim ketua dalam sidang pembacaan putusan di pengadilan tipikor padang. majelis hakim juga menjatuhkan pidana denda sebesar rp350 juta subsider empat bulan kurungan dan membayar uang pengganti sebesar rp1.754.979.804.\n\ndengan ketentuan apabila tidak dibayar paling lama satu bulan setelah putusan pengadilan memperoleh kekuatan hukum tetap (inkracht) maka hartanya akan disita dan dilelang.\n\nbaca juga: oknum asn jalani sidang perdana kasus penyelewengan infak masjid raya\n

In [ ]:
for i in df_content.index:
    tes = re.sub("\s\s+" , " ", df_content['konten_berita'][i])
    tes = re.sub(r'[^\w]', ' ', tes)
    tes = re.sub("\s\s+" , " ", tes)

    tokenize = word_tokenize(tes)
    tes = [word for word in tokenize if not word in stopwords.words('indonesian')]
    tes = " ".join(tes)
#     tes = stemmer.stem(tes)
    df_content['konten_berita'][i] = tes